In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import sklearn.preprocessing as skl_pre
import sklearn.linear_model as skl_lm
import sklearn.discriminant_analysis as skl_da
import sklearn.neighbors as skl_nb
import sklearn.model_selection as skl_ms

import warnings
warnings.filterwarnings('ignore') # Suppress skl k-NN param warnings 

from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, RandomForestClassifier


# This is a Markdown cell
Here we can write stuff, including LaTeX commands. To create one, go to the menu call at the top, then Cell $\rightarrow$ Cell Type $\rightarrow$ Markdown.

Here is a matrix
$$
\textbf{X} =  \begin{bmatrix}
    1 & 2 \\
    1 & 3 \\ 
\end{bmatrix}
$$

# 3 Data analysis task

The first step – before we start to build a model – is always to take a closer look at the data by
analyzing some statistics of the data set. In this step you can already gain some insights into the data
which helps you interpret the results of your methods later in the project.
Look into the provided data set by e.g. plotting the individual features. Based on your analysis,
answer the following questions:

- Do men or women dominate speaking roles in Hollywood movies?
- Has gender balance in speaking roles changed over time (i.e. years)?
- Do films in which men do more speaking make a lot more money than films in which women speak more?

Write 1-2 sentences for each question and support your findings with evidence (statistics, plots, ...).

# 4 Implementation of Methods
## 4.1 Methods to explore
The course has (so far) covered the five following ‘families’ of classification methods:

- logistic regression
- discriminant analysis: LDA, QDA
- K-nearest neighbor
- Tree-based methods: classification trees, random forests, bagging
- Boosting

In this project, you decide upon at least as many ‘families’ as you are group members, and decide in
each ‘family’ at least one method to explore. To be clear, **each group member should independently
implement and write about one method.** Who implemented which method should later be clearly
written in the contribution statement. All group members should be able to stand for all sections of
the report.

**Additionally**, you should include a naive classifier that always predicts male as the lead actor in your
analysis and compare its performance with your methods.

*Deep learning, which will be covered later, is also possible to use for classification. You are of course
welcome to explore this as well, in addition to the minimum requirements.*

## 4.2 What to do with each method
For *each* method you decide to explore, you should do the following:

1. Implement the method. We suggest that you use Python, and you may write your own code or use packages (the material from the problem solving sessions can be useful).
2. Tune the method to perform well.
3. Evaluate its performance using, e.g., cross validation. Exactly how to carry this out is up for you to decide.

Once you have completed the aforementioned tasks, you should with a good motivation (hint: cross
validation) select which method you decide to use ‘in production’ on a test set that will be made
available later. **Work on this part of the project together and write the results together.**